In [1]:
import os
import codecs
import pandas as pd

os.chdir('../')

In [2]:

from scripts.dataset_downloader import initialize_dataset
from scripts.k_means import create_clsuters, clustering_errors
from scripts.helpers import get_most_rated_movies
import pandas as pd
dataset = 'framework-ml-25m-subset(3) (EL2.4)'
#dataset = 'ml-25m-subset(3)-#2.2'
#dataset =  'ml-25m-subset(3)-nf-1-2-3-4'
#dataset = 'framework-ml-25m-subset'
#dataset_path = os.path.join('datasets', dataset)

In [3]:
# If dataset hasn't been previously initialized, it can be done with this function
#initialize_dataset(dataset)

In [4]:
os.getcwd()

'C:\\Users\\clari\\Desktop\\M2 - Thesis\\Research\\Dr Jacques Bou Abdo\\Recommender System\\5 - Ensemble Learning Model\\Accuracy Metrics\\serendipity-main'

In [5]:
import pandas as pd


In [6]:
genres_file = "datasets"


In [7]:
ratings_file =  "datasets"


In [8]:
import pandas as pd
dataset_name = dataset
#dataset_name = 'framework-ml-25m-subset'  # Adjust this as needed

base_path = f"C:\\Users\\clari\\Desktop\\M2 - Thesis\\Research\\Dr Jacques Bou Abdo\\Recommender System\\5 - Ensemble Learning Model\\Accuracy Metrics\\serendipity-main\\datasets\\{dataset_name}\\"

def load_data(dataset_name, base_path):
    ratings_path = f'{base_path}clean\\ratings.csv'
    movies_path = f'{base_path}movies.csv'
    
    ratings = pd.read_csv(ratings_path)
    movies = pd.read_csv(movies_path)
#    print(movies)
    if 'nf' in dataset_name or '25m-subset' in dataset_name:
        ratings = ratings.rename(columns={'itemId': 'movieId'})
        
        print(len(ratings))
        all_ratings = pd.read_csv(r"C:\Users\clari\Desktop\M2 - Thesis\Research\Dr Jacques Bou Abdo\Recommender System\4 - Review\dataset\ml-25m\Full Data set (Used with NF3 First)\ratings.csv")
        print(len(all_ratings))
        ratings = pd.merge(ratings, movies, on='movieId').sort_values(by='userId')
        print(len(ratings))
        print(ratings.columns)
        if ('nf1' in dataset_name) or ('nf2' in dataset_name) or ('nf4' in dataset_name):
            #ratings_detailed = pd.merge(ratings, all_ratings,left_on=['userId', 'movieId', 'rating_OLD'], right_on=['userId', 'movieId', 'rating']).sort_values(by='userId').rename(columns={'genres_y': 'genres', 'timestamp_x': 'timestamp', 'title_x': 'title','rating_x':'rating'})
             ratings_detailed = pd.merge(ratings, all_ratings, on=['userId', 'movieId', 'rating']).sort_values(by='userId').rename(columns={'genres_y': 'genres', 'timestamp_x': 'timestamp', 'title_x': 'title'})
        else:
            ratings_detailed = pd.merge(ratings, all_ratings, on=['userId', 'movieId', 'rating']).sort_values(by='userId').rename(columns={'genres_y': 'genres', 'timestamp_x': 'timestamp', 'title_x': 'title'})
        print(len(ratings_detailed))
    else:
        ratings = ratings.rename(columns={'itemId': 'movieId'})
        ratings_detailed = pd.merge(ratings, movies, on='movieId')
    
    return ratings_detailed,ratings


ratings_detailed,ratings = load_data(dataset_name, base_path)
print(len(ratings_detailed))

if '#2' in dataset_name:
    ratings_detailed = ratings_detailed[(ratings_detailed['isNoisy'] == "[0.]") | (ratings['isNoisy'] == "0")]
elif 'nf-1' in dataset_name:
    #ratings_detailed =  ratings_detailed[(ratings_detailed['1&2&3&4 = 0'] == 0) & (ratings_detailed['1&2&3&4 = 1'] == 0)]
    ratings_detailed =  ratings_detailed[(ratings_detailed['1&2&3&4 = 1'] == 0)]
#elif ('nf1' in dataset_name) or ('nf2' in dataset_name) or ('nf3' in dataset_name) or ('nf4' in dataset_name):
#    ratings_detailed = ratings_detailed

elif (not 'framework' in dataset_name) & ('ml-25m-subset(3)-' in dataset_name):
    ratings_detailed = ratings_detailed[ratings_detailed['isNoisy'] == 0]
elif 'framework' in dataset_name:
    ratings_detailed = ratings_detailed[ratings_detailed['layer3_result'] == 0]
# Uncomment if saving is required
# ratings_detailed.to_csv(f'datasets/{dataset_name}/clean/ratings.csv', index=False)

ratings_detailed.head()

634341
25000095
634341
Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'userId', 'movieId', 'rating',
       'timestamp', 'user_cat', 'rating_group', 'item_cat', 'nf1',
       'user_group', 'coherence', 'title_x', 'genres_x', 'RND', 'thresh',
       'nf2', 'Unnamed: 0', 'prediction', 'nf3', 'nf4', 'noisedegree', '1&2',
       '1&3', '1&4', '2&3', '2&4', '3&4', '1&2&3', '2&3&4', '1&2&4', '1&3&4',
       '1&2&3&4 = 0', '1&2&3&4 = 1', 'isNoisy', 'FinalOutput', 'OptOut',
       'layer3_result', 'title_y', 'genres_y'],
      dtype='object')
634341
634341


,Unnamed: 0.2,Unnamed: 0.1,userId,movieId,rating,timestamp,user_cat,rating_group,item_cat,nf1,...,1&3&4,1&2&3&4 = 0,1&2&3&4 = 1,isNoisy,FinalOutput,OptOut,layer3_result,title_y,genres,timestamp_y
36,39936,39936,107,2599,3.5,1112450353,Benevolent,Su,Strongly-preferred,0,...,0,0,0,0,0,0,0,Election (1999),Comedy,1112450353
38,515349,515349,107,7096,3.0,1116042392,Benevolent,Su,Strongly-preferred,0,...,0,0,0,0,0,0,0,Rivers and Tides (2001),Documentary,1116042392
39,177803,177803,107,337,3.5,1116042439,Benevolent,Su,Strongly-preferred,0,...,0,0,0,0,0,0,0,What's Eating Gilbert Grape (1993),Drama,1116042439
40,398955,398955,107,1258,3.5,1112450322,Benevolent,Su,Strongly-preferred,0,...,0,0,0,0,0,0,0,"Shining, The (1980)",Horror,1112450322
44,396544,396544,107,45028,3.5,1162221173,Benevolent,Su,Strongly-preferred,0,...,0,1,0,0,0,0,0,"Prairie Home Companion, A (2006)",Comedy|Drama|Musical,1162221173


In [9]:
ratings_detailed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107084 entries, 36 to 634325
Data columns (total 41 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0.2   107084 non-null  int64  
 1   Unnamed: 0.1   107084 non-null  int64  
 2   userId         107084 non-null  int64  
 3   movieId        107084 non-null  int64  
 4   rating         107084 non-null  float64
 5   timestamp      107084 non-null  int64  
 6   user_cat       107084 non-null  object 
 7   rating_group   107084 non-null  object 
 8   item_cat       107084 non-null  object 
 9   nf1            107084 non-null  int64  
 10  user_group     107084 non-null  object 
 11  coherence      107084 non-null  float64
 12  title          107084 non-null  object 
 13  genres_x       107084 non-null  object 
 14  RND            107084 non-null  float64
 15  thresh         107084 non-null  float64
 16  nf2            107084 non-null  int64  
 17  Unnamed: 0     107084 non-null  i

In [10]:
print(f'Number of ratings: {len(ratings_detailed)}')


Number of ratings: 107084


In [11]:
from clustering.helpers import transform_df
from clustering.kmeans import create_clsuters

# number of clusters
k = 100
ratings_detailed['user_cat'] = None
ratings_detailed['rating_group'] = None
ratings_detailed['item_cat'] = None
ratings_detailed['user_group'] = None
ratings_detailed['genres_x'] = None
ratings_detailed['timestamp_y'] = None
ratings_detailed['title_x'] = None
ratings_detailed['title_y'] = None
ratings_detailed['timestamp_x'] = None
ratings_detailed['genres_y'] = None
ratings_detailed['prediction'] = None
ratings_detailed['prediction_y'] = None
ratings_detailed['prediction_x'] = None



if 'nf1' in dataset_name:
        ratings_detailed = ratings_detailed.drop(columns=['user_cat', 'rating_group', 'item_cat', 'user_group', 'prediction','genres_y','genres_x','title_x','timestamp_x','title_y','timestamp_y','prediction_x','prediction_y'])
    #.rename(columns={'genres_x':'genres', 'title_x':'title','timestamp_x':'timestamp'})
 
elif ('#2' in dataset_name):
#or ('#2' in dataset_name) or ('#3' in dataset_name) :
    ratings_detailed['isNoisy'] = ratings_detailed['isNoisy'].apply(lambda x: x.strip('[]'))
    ratings_detailed = ratings_detailed.drop(columns=['user_cat', 'rating_group', 'item_cat', 'user_group', 'prediction','genres_y','genres_x','title_x','timestamp_x','title_y','timestamp_y','prediction_x','prediction_y'])

    #ratings_detailed['isNoisy'] = ratings_detailed['isNoisy'].strip('[]').astype(int).copy()  
elif dataset_name == 'ml-5m':
         ratings_detailed = ratings_detailed.drop(columns=['user_cat', 'rating_group', 'item_cat', 'user_group', 'prediction','genres_y','genres_x','title_x','timestamp_x','title_y','timestamp_y','prediction_x','prediction_y'])

else:
     ratings_detailed['prediction'] = None
     ratings_detailed = ratings_detailed.drop(columns=['user_cat', 'rating_group', 'item_cat', 'user_group', 'prediction','genres_y','genres_x','title_x','timestamp_x','title_y','timestamp_y','prediction_x','prediction_y'])
#ratings_detailed = ratings_detailed.drop(columns=['isNoisy'])
if ('EL2' in dataset_name) and ('framework' in dataset_name):
    ratings_detailed = ratings_detailed.drop(columns=['isNoisy'])
#ratings_detailed = ratings_detailed.copy().rename()
ratings_detailed.head()

,Unnamed: 0.2,Unnamed: 0.1,userId,movieId,rating,timestamp,nf1,coherence,title,RND,...,1&2&3,2&3&4,1&2&4,1&3&4,1&2&3&4 = 0,1&2&3&4 = 1,FinalOutput,OptOut,layer3_result,genres
36,39936,39936,107,2599,3.5,1112450353,0,17.145349,Election (1999),0.000000,...,0,0,0,0,0,0,0,0,0,Comedy
38,515349,515349,107,7096,3.0,1116042392,0,17.145349,Rivers and Tides (2001),0.192308,...,0,1,0,0,0,0,0,0,0,Documentary
39,177803,177803,107,337,3.5,1116042439,0,17.145349,What's Eating Gilbert Grape (1993),0.000000,...,0,0,0,0,0,0,0,0,0,Drama
40,398955,398955,107,1258,3.5,1112450322,0,17.145349,"Shining, The (1980)",0.000000,...,0,0,0,0,0,0,0,0,0,Horror
44,396544,396544,107,45028,3.5,1162221173,0,17.145349,"Prairie Home Companion, A (2006)",0.000000,...,0,0,0,0,1,0,0,0,0,Comedy|Drama|Musical


In [12]:
# use k means to cluster the users in the dataset

transformed_df = transform_df(ratings_detailed)
ratings_clustered = create_clsuters(k, transformed_df)
ratings_clustered

C:\Users\clari\Desktop\M2 - Thesis\Research\Dr Jacques Bou Abdo\Recommender System\5 - Ensemble Learning Model\Accuracy Metrics\serendipity-main\clustering\helpers.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataset.loc[dataset['genres'].str.contains(i), i] = 1


,userId,movieId,rating,timestamp,cluster
36,107,2599,3.5,1112450353,80
38,107,7096,3.0,1116042392,8
39,107,337,3.5,1116042439,8
40,107,1258,3.5,1112450322,80
44,107,45028,3.5,1162221173,30
...,...,...,...,...,...
634220,162476,457,4.5,1102095665,28
634217,162476,269,3.5,1102097883,28
634216,162476,4847,3.5,1102097826,28
634321,162488,1285,5.0,1202822575,45


In [13]:
ratings_clustered.to_csv('output/exp-4/' + dataset_name + '/clusters.csv', index=False)

In [14]:
otp = 'output/exp-4/'
name = '/means_output_clusters_CH.csv'
ratings_clustered.to_csv(otp + dataset + name, index=False)

OPTIONAL STEP// ADDED BY CLARITA, BECAUSE EXP3 EXPECTS COLUMNS THAT CAN ONLY BE GENERATED BY OTHER CLUSTERING METHODS
AND OTHER CLUSTERING METHODS TAKE INTO CONSIDERATION USER FILE, WHICH WE DONT HAVE IN 25M OF MOVIE LENS